In [4]:
""" This is Phase 1 project Movie Data Exploration File started on 6 May 2023"""

import pandas as pd
import numpy as np
import sqlite3
import os
import zlib
import zipfile
from zipfile import ZipFile






In [38]:
# Extract SQL db from zip
# ZipFile.extractall(file_path , members=None, pwd=None)Home\MovieData\


#textFile = zipfile.ZipFile("/home/hatenotwelcome/MovieData/im.db.zip", "r")
#textFile.close()

#SQL DB includes 8 tables: principals, known-for, directors, writers, persons, movie_basics, movie_ratings, movie_akas
# principals: *movie_id, 



conn = sqlite3.Connection("/home/bringingthesparkle/FlatIron/Movie_Data/im.db")


q = """
SELECT *
FROM principals
LIMIT 5
;
"""

pd.read_sql(q, conn)



,movie_id,ordering,person_id,category,job,characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
1,tt0111414,2,nm0398271,director,None,None
2,tt0111414,3,nm3739909,producer,producer,None
3,tt0323808,10,nm0059247,editor,None,None
4,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]"


In [48]:
#This is the CSV containing  movie names and their domestic grosses and foreign grosses
movie_grosses = pd.read_csv('/home/bringingthesparkle/FlatIron/Movie_Data/bom.movie_gross.csv.gz', sep=',')

print(movie_grosses.dtypes)
print(movie_grosses.tail(2))
print('\n')

print ("Missing values by Column")
for col in movie_grosses.columns:
    missing = movie_grosses[col].isna().sum()
    print (str(col) + ": " + str(missing) + " missing values")

movie_grosses = movie_grosses.replace(np.nan, 'Unknown')

for col in movie_grosses.columns:
    assert movie_grosses[col].isna().sum() == 0

print(movie_grosses.shape)

title              object
studio             object
domestic_gross    float64
foreign_gross      object
year                int64
dtype: object
                  title      studio  domestic_gross foreign_gross  year
3385           The Swan  Synergetic          2400.0           NaN  2018
3386  An Actor Prepares       Grav.          1700.0           NaN  2018


Missing values by Column
title: 0 missing values
studio: 5 missing values
domestic_gross: 28 missing values
foreign_gross: 1350 missing values
year: 0 missing values
(3387, 5)


In [ ]:
# creating new dataframe to represent only those with domestic gross, removing rows without
movie_grosses_domestic = movie_grosses.dropna(subset=['domestic_gross'])
assert movie_grosses_domestic == 
# then split/apply/combine on studio

In [6]:
# This is the CSV with movie information with columns: id, synopsis, rating, genre, director, writer, theater_date, dvd_date, currency, box_office, runtime, studio.
# ID has type int64, all others are mixed type objects and may need cleaning. 
movie_info = pd.read_csv('/home/bringingthesparkle/FlatIron/Movie_Data/rt.movie_info.tsv', sep='\t')
#print(movie_info.head(2))

#print(movie_info.dtypes)
#print(movie_info.shape) # Shape is (1560 by 12)

# Assert that 'id' is a unique record and there are no missing values
assert movie_info["id"].isna().sum() == 0

#Using code from Pandas Data Cleaning Lab to look for NaNs
print ("Missing values by Column")
for col in movie_info.columns:
    missing = movie_info[col].isna().sum()
    print (str(col) + ": " + str(missing) + " missing values")

#.sample(5, random_state=1)
#print(len(missing))  # There are 494 with studio listed, the rest have NaNs
#missing = movie_info[movie_info["id"].isna()]
#print(len(missing)) # There are 1498 with synopsis listed, the other 62 have NaNs.



Missing values by Column
id: 0 missing values
synopsis: 62 missing values
rating: 3 missing values
genre: 8 missing values
director: 199 missing values
writer: 449 missing values
theater_date: 359 missing values
dvd_date: 359 missing values
currency: 1220 missing values
box_office: 1220 missing values
runtime: 30 missing values
studio: 1066 missing values


In [7]:
# Replacing NaNs and asserting that none remain

movie_info = movie_info.replace(np.nan, 'Unknown')

for col in movie_info.columns:
    assert movie_info[col].isna().sum() == 0



In [8]:
movie_info["genre"].value_counts().nlargest(10)
# Clearly the genre column contains more than one category and would be best if separated into dictionaries where you can query on the key

genre
Drama                                              151
Comedy                                             110
Comedy|Drama                                        80
Drama|Mystery and Suspense                          67
Art House and International|Drama                   62
Action and Adventure|Drama                          42
Action and Adventure|Drama|Mystery and Suspense     40
Drama|Romance                                       35
Comedy|Romance                                      32
Art House and International|Comedy|Drama            31
Name: count, dtype: int64

In [37]:
# Massage the data 

# Create a list of all genres to query on.
movie_genres = []
movie_genre_dict = {}

for index, row in movie_info.iterrows():
    massage = row['genre']
    data = massage.split("|")
    #print (data)
    for word in data:
        if word in movie_genres:
            movie_genre_dict[word] +=1
        else:
            movie_genres.append(word)
            movie_genre_dict[word] = 1

    

print("There are " + str(len(movie_genres)) + " unique genres.")
print (movie_genres)

df = pd.DataFrame(movie_genre_dict.items(), columns=["Genre", "Genre_Counts"])
print (df.sort_values(by=['Genre_Counts'], ascending = False))


There are 22 unique genres.
['Action and Adventure', 'Classics', 'Drama', 'Science Fiction and Fantasy', 'Musical and Performing Arts', 'Mystery and Suspense', 'Romance', 'Kids and Family', 'Comedy', 'Unknown', 'Documentary', 'Special Interest', 'Art House and International', 'Horror', 'Western', 'Television', 'Sports and Fitness', 'Animation', 'Faith and Spirituality', 'Cult Movies', 'Anime and Manga', 'Gay and Lesbian']
                          Genre  Genre_Counts
2                         Drama           912
8                        Comedy           550
0          Action and Adventure           366
5          Mystery and Suspense           309
12  Art House and International           265
6                       Romance           198
1                      Classics           193
3   Science Fiction and Fantasy           172
13                       Horror           134
7               Kids and Family            99
4   Musical and Performing Arts            98
10                  Do